# Country Club Case Study
## Part 1 - PHPMyAdmin

**/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */**

SELECT name
FROM Facilities
WHERE membercost != 0;

**/* Q2: How many facilities do not charge a fee to members? */**

SELECT COUNT(facid) AS free_for_members
FROM Facilities
WHERE membercost = 0;

**/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */**

SELECT *
FROM
	(SELECT facid, 
		name, 
		membercost, 
		monthlymaintenance,
		(membercost / monthlymaintenance * 100) AS perc_mon_maint
    	FROM Facilities) AS subquery
WHERE perc_mon_maint < 20;

**/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */**

SELECT *
FROM Facilities
WHERE facid IN (1,5);

**/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100. Return the name and monthly maintenance of the facilities in question. */**

SELECT 
	name,
	monthlymaintenance,
	CASE WHEN monthlymaintenance > 100 THEN 'expensive'
	ELSE 'cheap' END AS price_desc
FROM Facilities;

**/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */**

SELECT firstname, surname
FROM Members
WHERE joindate = (SELECT MAX(joindate) FROM Members);

**/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name. */**

SELECT DISTINCT CONCAT_WS(' ', firstname, surname) AS full_name, name AS facility_name
FROM Bookings AS b
LEFT JOIN Facilities AS f
	USING(facid)
LEFT JOIN Members AS m
	USING(memid)
WHERE name LIKE 'Tennis Court%'
GROUP BY full_name;

_Note: Some members booked multiple courts. This query will only show one of the courts that the member booked and not both (which would result in the same member showing up multiple times in the table)._

**/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries. */**


SELECT 
	name AS facility_name,
	CASE WHEN firstname = 'GUEST' THEN 'GUEST'
		ELSE CONCAT_WS(' ',firstname, surname) END AS full_name,
	CASE WHEN memid = 0 THEN (slots * guestcost)
		ELSE (slots * membercost) END AS cost
FROM Bookings as b
LEFT JOIN Facilities as f
	USING(facid)
LEFT JOIN Members as m
	USING(memid)
WHERE starttime LIKE '2012-09-14%'
HAVING cost > 30
ORDER BY cost DESC;

**/* Q9: This time, produce the same result as in Q8, but using a subquery. */**

SELECT facility_name,
		CASE WHEN memid = 0 THEN 'GUEST'
			ELSE CONCAT_WS(' ', firstname,surname) END AS member_name,
		booking_cost
FROM
    (SELECT
		bookid,
    	b.memid,
 		name AS facility_name,
		CASE WHEN b.memid = 0 THEN slots * guestcost
			ELSE slots * membercost END AS booking_cost
	FROM Bookings AS b
	LEFT JOIN Facilities AS f
	USING(facid)
    WHERE starttime LIKE '2012-09-14%') AS mysub
LEFT JOIN Members
	USING(memid)
WHERE booking_cost > 30
ORDER BY booking_cost DESC;

## PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  



In [1]:
# Imports
import sqlite3
import pandas as pd

**/* Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members! */**

In [2]:
conn = sqlite3.connect('sqlite_db_pythonsqlite.db')

sql_query10 = pd.read_sql_query(
'''
SELECT 
    name AS facility_name,
	facid,
	SUM((CASE WHEN memid = 0 THEN slots * guestcost
     	ELSE slots * membercost END)) AS revenue
FROM Bookings
LEFT JOIN Facilities
	USING(facid)
GROUP BY facility_name
HAVING revenue < 1000
ORDER BY revenue;
''', conn)
query10_df = pd.DataFrame(sql_query10, columns = ['facility_name', 'facid', 'revenue'])

In [3]:
print(query10_df)

   facility_name  facid  revenue
0   Table Tennis      3      180
1  Snooker Table      7      240
2     Pool Table      8      270


**/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */**

_Note: This query filters out members that don't have a recommender._

In [4]:
sql_query11 = pd.read_sql_query(
'''
SELECT m1.surname,
    m1.firstname,
    m2.firstname || ' '|| m2.surname AS recommender
FROM Members as m1
INNER JOIN Members as m2
  ON m1.recommendedby = m2.memid
WHERE m1.memid != 0
ORDER BY m1.surname, m1.firstname;
''', conn)
query11_df = pd.DataFrame(sql_query11, columns = ['surname', 'firstname', 'recommender'])

In [5]:
print(query11_df)

              surname  firstname        recommender
0               Bader   Florence    Ponder Stibbons
1               Baker       Anne    Ponder Stibbons
2               Baker    Timothy     Jemima Farrell
3              Boothe        Tim         Tim Rownam
4             Butters     Gerald       Darren Smith
5              Coplin       Joan      Timothy Baker
6             Crumpet      Erica        Tracy Smith
7                Dare      Nancy    Janice Joplette
8             Genting    Matthew     Gerald Butters
9                Hunt       John  Millicent Purview
10              Jones      David    Janice Joplette
11              Jones    Douglas        David Jones
12           Joplette     Janice       Darren Smith
13          Mackenzie       Anna       Darren Smith
14               Owen    Charles       Darren Smith
15             Pinker      David     Jemima Farrell
16            Purview  Millicent        Tracy Smith
17             Rumney  Henrietta    Matthew Genting
18          

/* Q12: Find the facilities with their usage by member, but not guests */

In [38]:
sql_query12 = pd.read_sql_query(
'''
SELECT
    facid,
    name as facility_name,
    SUM(CASE WHEN memid = 0 THEN 0
            ELSE slots END) AS usage
FROM Bookings
LEFT JOIN Facilities
  USING(facid)
GROUP BY facid, facility_name
ORDER BY usage DESC;
''', conn)
query12_df = pd.DataFrame(sql_query12, columns = ['facid', 'facility_name', 'usage'])

In [39]:
print(query12_df)

   facid    facility_name  usage
0      2  Badminton Court   1086
1      0   Tennis Court 1    957
2      4   Massage Room 1    884
3      1   Tennis Court 2    882
4      7    Snooker Table    860
5      8       Pool Table    856
6      3     Table Tennis    794
7      6     Squash Court    418
8      5   Massage Room 2     54


**/* Q13: Find the facilities usage by month, but not guests */**

strftime('%m', starttime) <- to extract month.

In [44]:
sql_query13 = pd.read_sql_query(
'''
SELECT
    STRFTIME('%m', starttime) AS month,
    SUM(CASE WHEN memid = 0 THEN 0
            ELSE slots END) AS usage
FROM Bookings
LEFT JOIN Facilities
  USING(facid)
GROUP BY month
ORDER BY month, usage DESC;
''', conn)
query13_df = pd.DataFrame(sql_query13, columns = ['month', 'usage'])

In [45]:
print(query13_df)

  month  usage
0    07   1061
1    08   2531
2    09   3199
